In [9]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torch.utils.data.sampler import SubsetRandomSampler
from torchsummary import summary

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
print(device)

cuda


In [35]:
#Prepare the train and valid preprocessing code
def train_valid_loader(data_dir,
                       batch_size,
                       augment,
                       random_seed,
                       valid_size=0.1,
                       shuffle = True):
    normalize = transforms.Normalize(
        mean = [0.4914, 0.4822, 0.4465],
        std = [0.2023, 0.1994, 0.2010]
    )
    #Define required transforms
    valid_transform = transforms.Compose([
        transforms.Resize((227, 227)),
        transforms.ToTensor(),
        normalize
    ])
    if augment:
        train_transform = transforms.Compose([
            transforms.RandomCrop(32,padding = 4),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            normalize
        ])
    else:
        train_transform = transforms.Compose([
            transforms.Resize((227, 227)),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            normalize
        ])

    #Load the dataset
    train_dataset = datasets.CIFAR10(
        root = data_dir,
        train = True,
        download = True,
        transform = train_transform,)
    valid_dataset = datasets.CIFAR10(
        root = data_dir,
        train = True,
        download = True,
        transform = valid_transform,)

    #Get parameters
    num_train = len(train_dataset)
    indices = list(range(num_train))
    split = int(np.floor(valid_size * num_train))

    #Shuffle indices
    if shuffle:
        np.random.seed(random_seed)
        np.random.shuffle(indices)

    #Split indices
    train_idx, valid_idx = indices[split:], indices[:split]

    #Sampler
    train_sampler = SubsetRandomSampler(train_idx)
    valid_sampler = SubsetRandomSampler(valid_idx)

    #Loaders
    train_loader = DataLoader(
        train_dataset,
        batch_size = batch_size,
        sampler = train_sampler,)
    valid_loader = DataLoader(
        valid_dataset,
        batch_size = batch_size,
        sampler = valid_sampler,)

    return (train_loader, valid_loader)


In [36]:
def test_loader(data_dir, batch_size,shuffle=True):
    normalize = transforms.Normalize(
        mean = [0.4914, 0.4822, 0.4465],
        std = [0.2023, 0.1994, 0.2010],
    )

    #Define transforms
    transform = transforms.Compose([
        transforms.Resize((227, 227)),
        transforms.ToTensor(),
        normalize,])

    #Create dataset
    dataset = datasets.CIFAR10(
        root = data_dir,
        train = False,
        download = True,
        transform = transform,)

    #Create dataloader
    loader = DataLoader(
        dataset,
        batch_size = batch_size,
        shuffle = shuffle,)

    return loader


In [37]:
train_loader, valid_loader = train_valid_loader(
    data_dir = 'data',
    batch_size = 64,
    augment = False,
    random_seed = 42,)

test_loader = test_loader(
    data_dir = 'data',
    batch_size = 64,)


Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


In [38]:
#Create skeleton code for the class
class AlexNet(nn.Module):
    def __init__(self, num_classes = 10):
        super(AlexNet, self).__init__()
        self.l1 = nn.Sequential(
            nn.Conv2d(3,96,kernel_size=11,stride=4,padding=0),
            nn.BatchNorm2d(96),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3,stride=2)
        )
        self.l2 = nn.Sequential(
            nn.Conv2d(96,256,kernel_size=5,stride=1,padding=2),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3,stride=2)
        )
        self.l3 = nn.Sequential(
            nn.Conv2d(256,384,kernel_size=3,stride=1,padding=1),
            nn.BatchNorm2d(384),
            nn.ReLU(),
        )
        self.l4 = nn.Sequential(
            nn.Conv2d(384,384,kernel_size=3,stride=1,padding=1),
            nn.BatchNorm2d(384),
            nn.ReLU(),
        )
        self.l5 = nn.Sequential(
            nn.Conv2d(384,256,kernel_size=3,stride=1,padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3,stride=2)
        )
        self.fc = nn.Sequential(
            nn.Dropout(p=0.5),
            nn.Linear(9216,4096),
            nn.ReLU(),
        )
        self.fc1 = nn.Sequential(
            nn.Dropout(p=0.5),
            nn.Linear(4096,4096),
            nn.ReLU(),
        )
        self.fc2 = nn.Linear(4096,num_classes)

    def forward(self,x):
      out = self.l1(x)
      out = self.l2(out)
      out = self.l3(out)
      out = self.l4(out)
      out = self.l5(out)
      out = out.reshape(out.size(0),-1)
      out = self.fc(out)
      out = self.fc1(out)
      out = self.fc2(out)
      return out

In [39]:
summary(AlexNet().to(device),input_size=(3,227,227))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 96, 55, 55]          34,944
       BatchNorm2d-2           [-1, 96, 55, 55]             192
              ReLU-3           [-1, 96, 55, 55]               0
         MaxPool2d-4           [-1, 96, 27, 27]               0
            Conv2d-5          [-1, 256, 27, 27]         614,656
       BatchNorm2d-6          [-1, 256, 27, 27]             512
              ReLU-7          [-1, 256, 27, 27]               0
         MaxPool2d-8          [-1, 256, 13, 13]               0
            Conv2d-9          [-1, 384, 13, 13]         885,120
      BatchNorm2d-10          [-1, 384, 13, 13]             768
             ReLU-11          [-1, 384, 13, 13]               0
           Conv2d-12          [-1, 384, 13, 13]       1,327,488
      BatchNorm2d-13          [-1, 384, 13, 13]             768
             ReLU-14          [-1, 384,

In [40]:
num_classes = 10
num_epochs = 20
learning_rate = 0.005
batch_size = 64

model = AlexNet(num_classes).to(device)

#Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(),lr=learning_rate,momentum=0.9,weight_decay=0.005)

#Train the model
total_step = len(train_loader)
loss_list = []
val_acc_list = []

for epoch in range(num_epochs):
  for i,(images,labels) in enumerate(train_loader):
    images = images.to(device)
    labels = labels.to(device)

    #Forward pass
    outputs = model(images)
    loss = criterion(outputs,labels)

    #Backward and optimize
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
  print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))
  loss_list.append(loss.item())

  #Validation
  with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in valid_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        del images, labels, outputs
    val_acc_list.append(100 * correct / total)
    print('Accuracy of the network on the {} validation images: {} %'.format(5000, 100 * correct / total))

Epoch [1/20], Step [704/704], Loss: 1.2861
Accuracy of the network on the 5000 validation images: 60.16 %
Epoch [2/20], Step [704/704], Loss: 1.0192
Accuracy of the network on the 5000 validation images: 69.44 %
Epoch [3/20], Step [704/704], Loss: 0.5147
Accuracy of the network on the 5000 validation images: 72.44 %
Epoch [4/20], Step [704/704], Loss: 1.0825
Accuracy of the network on the 5000 validation images: 74.34 %
Epoch [5/20], Step [704/704], Loss: 0.4349
Accuracy of the network on the 5000 validation images: 77.5 %
Epoch [6/20], Step [704/704], Loss: 0.5414
Accuracy of the network on the 5000 validation images: 78.62 %
Epoch [7/20], Step [704/704], Loss: 0.8947
Accuracy of the network on the 5000 validation images: 78.74 %
Epoch [8/20], Step [704/704], Loss: 0.9373
Accuracy of the network on the 5000 validation images: 80.38 %
Epoch [9/20], Step [704/704], Loss: 0.6834
Accuracy of the network on the 5000 validation images: 79.84 %
Epoch [10/20], Step [704/704], Loss: 1.1152
Acc